## Conservación de la energía_new

El objetivo es implementar el módulo de conservación de la energía copiando lo que hace Gabriela en C++

Consideraciones:  
* En el código de Gabriela el vector y tiene: y[1]-> radio, y[2]-> velocidad, y[3]-> temperatura (T_bc), y[4]-> nh2o, y[5]-> TLi
* Uso el modelo de Toegel (modelo 2) solamente para la parte de conductividad térmica

A diferencia del otro notebook de conservacion_energia, acá copié todo desde cero y estuve intentando hacer los reemplazos todos juntos para no tener ningún error


In [ ]:
#Código necesario para importar funciones de otros notebooks .ipynb

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

#Ejemplo
# from derivada_reacciones import *

In [ ]:
import numpy as np
import math
pi = math.pi


from funciones_math import Vol, erf
from funciones_qcas import n_tot, n_gas
from presion import ctes_VanderWaals, ctes_VanderWaals_deriv, volumen_molar_deriv

from parametros import sigmap, Na, Cvh2o, Rg, var_termo, sigma, mu, rol, K, modelo, n_Yasui, Tinf, n_reacc, masa_species_Kgmol, alfaM, pvap0, n_species, ap, alfae, rapidez, Rv, cl, Pinf, masa_H2O_kgmol

from conductividad_termica import Conductividad_termica_k, Conductividad1, Conductividad2

from reacciones_qcas import lista_de_reacciones

In [ ]:
def derivada_radio_energia_new( y, n, dydt, dndt, nh2o0):




   #Asigno variables termodinámicas
   R = y[var_termo["R"]]
   v_R = y[var_termo["v_R"]]
   M = y[var_termo["m"]] #masa total
   T_bi = y[var_termo["T_bi"]]
   T_bc = y[var_termo["T_bc"]]
   T_li = y[var_termo["T_li"]]
   mp = y[var_termo["mp"]]


   V = Vol(R)

   ntot = n_tot(n)

   m=M/ntot #masa media de una particula

   landa=V/(np.sqrt(2.0)*sigmap*ntot)  #camino libre medio

   v=V/ntot*Na #volumen molar

   np_=ntot/V #densidad de particulas

   eh2o=Cvh2o/Na*T_bi #energia llevada por una molecula que se esta evaporando o codensando

   ngas = n_gas(n) #numero de particulas de no-condensables

   a,b = ctes_VanderWaals(ngas, ngas, ntot)

   # a = a_Ar*(ngas/ntot)*(ngas/ntot)+2.0*a_Arh2o*(ngas/ntot)*(y[Nvar2+6]/ntot)+
   #       a_h2o*(y[Nvar2+6]/ntot)*(y[Nvar2+6]/ntot);
   # #const a de VW promediada con la cantidad de particulas

   # b = b_Ar*(ngas/ntot)*(ngas/ntot)+2.0*b_Arh2o*(ngas/ntot)*(y[Nvar2+6]/ntot)+
   #       b_h2o*(y[Nvar2+6]/ntot)*(y[Nvar2+6]/ntot);
   # #const b de VW promediada con la cantidad de particulas

   pg=Rg*T_bc/(v-b)-a/(v*v) #presion total

   rog=M/V #densidad de la mezcla

   pB=pg-2.0*sigma/R-4.0*mu/R*(v_R-mp/rol)-mp*mp*(1.0/rol-1.0/rog)
   #presion en la interfase del lado del liquido

   pv=n[5]/ntot*pg #presion parcial del vapor de agua

   #ldiff longitud de difusion

   #***************************************************************************
   #************bloque que calcula el cambio debido a la difusion***************
   #****************************************************************************

   # y0[1]=nh20;
   # y0[2]=nh0;
   # y0[3]=no0;
   # y0[4]=no20;
   # y0[5]=noh0;
   # y0[6]=nh2o0;
   # y0[7]=nn0;
   # y0[8]=nnh0;
   # y0[9]=nnh20;
   # y0[10]=nnh30;
   # y0[11]=nn2h0;
   # y0[12]=nno0;
   # y0[13]=nno20;
   # y0[14]=nn2o0;
   # y0[15]=nhno0;
   # y0[16]=nn20;
   # y0[17]=nar0;
   # y0[18]=nh2o20;
   # y0[19]=nho20;

   # DiamColVect(DiamCol);


   # for(iw=1;iw<20;iw++){
   #    InvDi[iw]=0.0;

   #    for(j=1;j<20;j++){

   #       if((y[Nvar2+iw] != 0) && (y[Nvar2+j] != 0))
   #          InvDi[iw]+=(y[Nvar2+j]/ntot)/(1.0-y[Nvar2+iw]/ntot)/
   #          (3.0/8.0*np.sqrt(abs(pi*K*Tinf/(2.0*Masa[iw]*Masa[j]/
   #          (Masa[iw]+Masa[j])))/
   #          ((y[Nvar2+iw]+y[Nvar2+j])/V*pi*0.5*(DiamCol[iw]+DiamCol[j])*
   #          0.5*(DiamCol[iw]+DiamCol[j]))));

   #    }

   #    if((y[Nvar2+iw] != 0))
   #       InvDi[iw]-=(y[Nvar2+iw]/ntot)/(1.0-y[Nvar2+iw]/ntot)/
   #       (3.0/8.0*np.sqrt(abs(pi*K*Tinf/(2.0*Masa[iw]*Masa[iw]/
   #       (Masa[iw]+Masa[iw])))/
   #       ((y[Nvar2+iw]+y[Nvar2+iw])/V*pi*0.5*(DiamCol[iw]+DiamCol[iw])*
   #       0.5*(DiamCol[iw]+DiamCol[iw]))));

   #    if(InvDi[iw]<1.0e-9)
   #       InvDi[iw]=0.0;

   # }


   # for(i=1;i<20;i++){

   #    if(InvDi[i] != 0){
   #       if(v_R==0.0)
   #          ldiffi[i]=R/pi;
   #       else{
   #          if(np.sqrt(R/InvDi[i]/abs(v_R)) < (R/pi))
   #             ldiffi[i]=np.sqrt(R/InvDi[i]/abs(v_R));
   #          else
   #             ldiffi[i]=R/pi;
   #       }
   #    }

   #    else
   #       ldiffi[i]=R/pi;


   #    if(yndif != 0){
   #       if(InvDi[i] != 0)
   #          dydt[Nvar2+i]=4.0*pi*R*R/InvDi[i]*(y0[i]/(4.0*pi/3.0*Ri*Ri*Ri)-
   #          y[Nvar2+i]/(4.0*pi/3.0*R*R*R))/ldiffi[i];
   #       else
   #          dydt[Nvar2+i]=0.0;
   #    }
   #    else
   #       dydt[Nvar2+i]=0.0;      

   # }

   #***************************************************************************
   #***************************************************************************
   #***************************************************************************

   #Antes de calcular la variacion de particulas por disociacion, necesito solo
   #por difusion

   # fraccion(fracc);

   CalorDifusion=0.0

   # for(i=1;i<20;i++)
   #    CalorDifusion=CalorDifusion+(1.0+fracc[i]/2.0)*K*Tinf*dydt[Nvar2+i];


   #***************************************************************************
   #***********bloque que calcula disociacion de las distintas especies********
   #***************************************************************************

   #y[Nvar2+1] =H2
   #y[Nvar2+2] =H
   #y[Nvar2+3] =O
   #y[Nvar2+4] =O2
   #y[Nvar2+5] =OH
   #y[Nvar2+6] =H2O = y[4] = vapor
   #y[Nvar2+7] =N
   #y[Nvar2+8] =NH
   #y[Nvar2+9] =NH2
   #y[Nvar2+10] =NH3
   #y[Nvar2+11] =N2H
   #y[Nvar2+12] =NO
   #y[Nvar2+13] =NO2
   #y[Nvar2+14] =N2O
   #y[Nvar2+15] =HNO
   #y[Nvar2+16] =N2
   #y[Nvar2+17] =Ar (gas noble)
   #y[Nvar2+18] =H2O2
   #y[Nvar2+19] =HO2


   # VolExcl=exp(ntot/V*MolCov/(1.0-ntot/V*MolCov))/(1.0-ntot/V*MolCov);

   # #rfi: reacciones forward para la especie Nvar2+i
   # #rbi: reacciones backward para la especie Nvar2+i
   # reacciones(y,VolExcl,ntot,V,Nvar2,r);

   # if(yndisoc !=0){
   #    rf1=r[0];
   #    rb1=r[1];
   #    rf2=r[2];
   #    rb2=r[3];
   #    rf3=r[4];
   #    rb3=r[5];
   #    rf4=r[6];
   #    rb4=r[7];
   #    rf5=r[8];
   #    rb5=r[9];
   #    rf6=r[10];
   #    rb6=r[11];
   #    rf7=r[12];
   #    rb7=r[13];
   #    rf8=r[14];
   #    rb8=r[15];
   #    rf9=r[16];
   #    rb9=r[17];
   #    rf10=r[18];
   #    rb10=r[19];
   #    rf11=r[20];
   #    rb11=r[21];
   #    rf12=r[22];
   #    rb12=r[23];
   #    rf13=r[24];
   #    rb13=r[25];
   #    rf14=r[26];
   #    rb14=r[27];
   #    rf15=r[28];
   #    rb15=r[29];
   #    rf16=r[30];
   #    rb16=r[31];
   #    rf17=r[32];
   #    rb17=r[33];
   #    rf18=r[34];
   #    rb18=r[35];
   #    rf19=r[36];
   #    rb19=r[37];
   #    rf20=r[38];
   #    rb20=r[39];
   #    rf21=r[40];
   #    rb21=r[41];
   #    rf22=r[42];
   #    rb22=r[43];
   #    rf23=r[44];
   #    rb23=r[45];
   #    rf24=r[46];
   #    rb24=r[47];
   #    rf25=r[48];
   #    rb25=r[49];
   #    rf26=r[50];
   #    rb26=r[51];
   #    rf27=r[52];
   #    rb27=r[53];
   #    rf28=r[54];
   #    rb28=r[55];
   #    rf29=r[56];
   #    rb29=r[57];
   #    rf30=r[58];
   #    rb30=r[59];
   #    rf31=r[60];
   #    rb31=r[61];
   #    rf32=r[62];
   #    rb32=r[63];
   #    rf33=r[64];
   #    rb33=r[65];
   #    rf34=r[66];
   #    rb34=r[67];
   #    rf35=r[68];
   #    rb35=r[69];
   #    rf36=r[70];
   #    rb36=r[71];
   #    rf37=r[72];
   #    rb37=r[73];
   #    rf38=r[74];
   #    rb38=r[75];
   #    rf39=r[76];
   #    rb39=r[77];
   #    rf40=r[78];
   #    rb40=r[79];
   #    rf41=r[80];
   #    rb41=r[81];
   #    rf42=r[82];
   #    rb42=r[83];
   #    rf43=r[84];
   #    rb43=r[85];
   #    rf44=r[86];
   #    rb44=r[87];
   #    rf45=r[88];
   #    rb45=r[89];
   #    rf46=r[90];
   #    rb46=r[91];
   #    rf47=r[92];
   #    rb47=r[93];
   #    rf48=r[94];
   #    rb48=r[95];
   #    rf49=r[96];
   #    rb49=r[97];
   #    rf50=r[98];
   #    rb50=r[99];
   #    rf51=r[100];
   #    rb51=r[101];

   #    #************************************************************************
   #    #*******************derivadas de las especies****************************
   #    #************************difusion + disociacion**************************


   #    dydt[Nvar2+1]=dydt[Nvar2+1]+V*(-1.0*(rf3-rb3)+1.0*(rf5-rb5)-1.0*(rf7-rb7)+1.0*(rf20-rb20)+
   #    1.0*(rf26-rb26)+1.0*(rf31-rb31)+1.0*(rf37-rb37)+1.0*(rf41-rb41)+1.0*
   #    (rf43-rb43)+1.0*(rf49-rb49));

   #    dydt[Nvar2+2]=dydt[Nvar2+2]+V*(-1.0*(rf2-rb2)+1.0*(rf3-rb3)-1.0*(rf4-rb4)-2.0*(rf5-rb5)-
   #    1.0*(rf6-rb6)+1.0*(rf7-rb7)+1.0*(rf11-rb11)-1.0*(rf14-rb14)-1.0*(rf18-rb18)+
   #    1.0*(rf19-rb19)-1.0*(rf20-rb20)+1.0*(rf21-rb21)+1.0*(rf25-rb25)+1.0*(rf28-rb28)+
   #    +1.0*(rf30-rb30)-1.0*(rf31-rb31)+1.0*(rf33-rb33)+1.0*(rf34-rb34)-1.0*(rf37-rb37)-
   #    1.0*(rf39-rb39)-1.0*(rf41-rb41)-1.0*(rf43-rb43)-1.0*(rf48-rb48)-1.0*(rf49-rb49));

   #    dydt[Nvar2+3]=dydt[Nvar2+3]+V*(-2.0*(rf1-rb1)-1.0*(rf2-rb2)-1.0*(rf3-rb3)+1.0*(rf4-rb4)
   #    +1.0*(rf8-rb8)+1.0*(rf9-rb9)+1.0*(rf10-rb10)-1.0*(rf12-rb12)-1.0*(rf13-rb13)+
   #    1.0*(rf15-rb15)-1.0*(rf16-rb16)-1.0*(rf17-rb17)-1.0*(rf19-rb19)+1.0*
   #    (rf23-rb23)-1.0*(rf29-rb29)-1.0*(rf30-rb30)-1.0*(rf35-rb35)-1.0*(rf36-rb36)
   #    -1.0*(rf40-rb40)-1.0*(rf45-rb45)-1.0*(rf50-rb50));

   #    dydt[Nvar2+4]=dydt[Nvar2+4]+V*(1.0*(rf1-rb1)-1.0*(rf4-rb4)-1.0*(rf10-rb10)+1.0*(rf12-rb12)+
   #    1.0*(rf17-rb17)-1.0*(rf23-rb23)-1.0*(rf24-rb24)+1.0*(rf46-rb46));

   #    dydt[Nvar2+5]=dydt[Nvar2+5]+V*(+1.0*(rf2-rb2)+1.0*(rf3-rb3)+1.0*(rf4-rb4)-1.0*(rf6-rb6)
   #    -1.0*(rf7-rb7)-2.0*(rf8-rb8)-1.0*(rf11-rb11)+1.0*(rf14-rb14)+1.0*(rf18-rb18)
   #    -1.0*(rf21-rb21)-1.0*(rf22-rb22)+1.0*(rf24-rb24)+1.0*(rf27-rb27)+1.0*(rf29-rb29)
   #    -1.0*(rf32-rb32)+1.0*(rf35-rb35)-1.0*(rf38-rb38)+1.0*(rf40-rb40)-1.0*(rf42-rb42)
   #    -1.0*(rf44-rb44)+1.0*(rf45-rb45)+2.0*(rf47-rb47)+1.0*(rf48-rb48)+1.0*(rf50-rb50));

   #    dydt[Nvar2+6]=dydt[Nvar2+6]+V*(+1.0*(rf6-rb6)+1.0*(rf7-rb7)+1.0*(rf8-rb8)+1.0*(rf22-rb22)
   #    -1.0*(rf26-rb26)+1.0*(rf32-rb32)+1.0*(rf38-rb38)+1.0*(rf42-rb42)+1.0*(rf44-rb44)+
   #    1.0*(rf48-rb48)+1.0*(rf51-rb51));

   #    /*
   #    dydt[Nvar2+7]=dydt[Nvar2+7]+V*(-1.0*(rf9-rb9)-1.0*(rf10-rb10)-1.0*(rf11-rb11)+1.0*(rf20-rb20)+1.0*(rf22-rb22)
   #    -1.0*(rf25-rb25));

   #    dydt[Nvar2+8]=dydt[Nvar2+8]+V*(-1.0*(rf19-rb19)-1.0*(rf20-rb20)-1.0*(rf21-rb21)-1.0*(rf22-rb22)
   #    -1.0*(rf23-rb23)-1.0*(rf24-rb24)-1.0*(rf25-rb25)-1.0*(rf26-rb26)-1.0*
   #    (rf27-rb27)-1.0*(rf28-rb28)+1.0*(rf29-rb29)+1.0*(rf31-rb31)+1.0*(rf32-rb32)
   #    +1.0*(rf36-rb36));

   #    dydt[Nvar2+9]=dydt[Nvar2+9]+V*(-1.0*(rf29-rb29)-1.0*(rf30-rb30)-1.0*(rf31-rb31)-1.0*(rf32-rb32)
   #    +1.0*(rf43-rb43)+1.0*(rf44-rb44)+1.0*(rf45-rb45));

   #    dydt[Nvar2+10]=dydt[Nvar2+10]+V*(-1.0*(rf43-rb43)-1.0*(rf44-rb44)-1.0*(rf45-rb45));

   #    dydt[Nvar2+11]=dydt[Nvar2+11]+V*(-1.0*(rf33-rb33)-1.0*(rf34-rb34)-1.0*(rf35-rb35)-1.0*(rf36-rb36)
   #    -1.0*(rf37-rb37)-1.0*(rf38-rb38));

   #    dydt[Nvar2+12]=dydt[Nvar2+12]+V*(-1.0*(rf9-rb9)+1.0*(rf10-rb10)+1.0*(rf11-rb11)+2.0*(rf13-rb13)
   #    -1.0*(rf16-rb16)+1.0*(rf17-rb17)+1.0*(rf18-rb18)+1.0*(rf19-rb19)+1.0*(rf24-rb24)
   #    -1.0*(rf27-rb27)-1.0*(rf28-rb28)+1.0*(rf36-rb36)-1.0*(rf39-rb39)+1.0*(rf40-rb40)
   #    +1.0*(rf41-rb41)+1.0*(rf42-rb42));

   #    dydt[Nvar2+13]=dydt[Nvar2+13]+V*(+1.0*(rf16-rb16)-1.0*(rf17-rb17)-1.0*(rf18-rb18));

   #    dydt[Nvar2+14]=dydt[Nvar2+14]+V*(-1.0*(rf12-rb12)-1.0*(rf13-rb13)-1.0*(rf14-rb14)-1.0*(rf15-rb15)+
   #    1.0*(rf28-rb28));

   #    dydt[Nvar2+15]=dydt[Nvar2+15]+V*(+1.0*(rf21-rb21)+1.0*(rf23-rb23)+1.0*(rf26-rb26)+1.0*(rf30-rb30)
   #    +1.0*(rf39-rb39)-1.0*(rf40-rb40)-1.0*(rf41-rb41)-1.0*(rf42-rb42));

   #    dydt[Nvar2+16]=dydt[Nvar2+16]+V*(+1.0*(rf9-rb9)+1.0*(rf12-rb12)+1.0*(rf14-rb14)+1.0*(rf15-rb15)+
   #    1.0*(rf25-rb25)+1.0*(rf27-rb27)+1.0*(rf33-rb33)+1.0*(rf34-rb34)+1.0*
   #    (rf35-rb35)+1.0*(rf37-rb37)+1.0*(rf38-rb38));
   #    */


   #    dydt[Nvar2+17]=dydt[Nvar2+17];

   #    dydt[Nvar2+18]=dydt[Nvar2+18]+V*(-1.0*(rf47-rb47)-1.0*(rf48-rb48)-1.0*(rf49-rb49)-1.0*(rf50-rb50)-
   #    1.0*(rf51-rb51)+1.0*(rf46-rb46));

   #    dydt[Nvar2+19]=dydt[Nvar2+19]+V*(-2.0*(rf46-rb46)+1.0*(rf49-rb49)+1.0*(rf50-rb50)+1.0*(rf51-rb51));

   # }

   # dydt[Nvar2+6]=dydt[Nvar2+6]+4.0*pi*R*R*(mp*Na/masa_H2O_kgmol); #variacion de la cantidad de particulas
   # #de vapor de agua debido a la condensacion, mas por difusion y reacciones quimicas

   dntdt = np.sum(dndt) #Esto tiene que venir después de las reacciones qcas

   dadt, dbdt = ctes_VanderWaals_deriv()
   dvdt =  volumen_molar_deriv()

   # dadt=-2.0/pow(ntot,3.0)*(a_Ar*ngas*ngas+2.0*a_Arh2o*ngas*y[Nvar2+6]-a_Arh2o*ngas*ntot+
   # a_h2o*y[Nvar2+6]*y[Nvar2+6]-a_h2o*y[Nvar2+6]*ntot)*dntdt;
   # #derivada respecto del tiempo de la const. a de VW

   # dbdt=-2.0/pow(ntot,3.0)*(b_Ar*ngas*ngas+2.0*b_Arh2o*ngas*y[Nvar2+6]-b_Arh2o*ngas*ntot+
   # b_h2o*y[Nvar2+6]*y[Nvar2+6]-b_h2o*y[Nvar2+6]*ntot)*dntdt;
   # #derivada respecto del tiempo de la const. b de VW

   # dvdt=-V*Na/(ntot*ntot)*dntdt+4.0*pi*R*R*Na/ntot*v_R;
   # #derivada del volumen molar con respecto a t

   k = Conductividad_termica_k(T_bc,n)
   Xi=k/(8.0/2.0*nh2o0*K+5.0/2.0*ngas*K)

   if(v_R == 0):
      lth=R/pi
   else:
      if((R/pi)<(np.sqrt(R*Xi/abs(v_R)))):
         lth=R/pi
      else:
         lth=np.sqrt(R*Xi/abs(v_R))



   if(modelo == 1): #Yasui
      qpp=k*(T_bi-T_bc)/n_Yasui/landa

   if(modelo == 2): #Toegel
      qpp=k*(Tinf-T_bc)/lth

   # if(modelo == 3){ #Hao-Prosperetti
   #    cont=NCh;
   #    qpp=0.;

   #    while(cont >= 0){
   #       DPTCh=DerPCheb(1.0,2.0*cont);
   #       qpp=qpp+y[cont+6]*DPTCh;
   #       cont=cont-1;
   #    }

   #    qpp=qpp/R;

   # }

   # if(modelo == 4){
   #    cont=NCh;
   #    qpp=0.;

   #    while(cont >= 2){
   #       DPTCh=DerPCheb(1.0,2.0*cont);
   #       qpp=qpp+y[cont+6]*DPTCh;
   #       cont=cont-1;
   #    }

   #    qpp=qpp/R+k*(Tinf-T_bc)/lth;;


   # }


   r = np.zeros(n_reacc)
   lista_de_reacciones(y, n, r)


   yndisoc = 1 #Lo pongo para que corra lo siguiente
   if(yndisoc != 0):
      CalorDisociacion=1.0e3*V/Na*((r[0]-r[1])*498.0+     (r[2]-r[3])*428.0+(r[4]-r[5])*(-8.0)+(r[6]-r[7])*(-70.0)+      (r[8]-r[9])*436.0+(r[10]-r[11])*498.0+(r[12]-r[13])*62.0+(r[14]-r[15])*70.0+(r[16]-r[17])*(175.35)+(r[18]-r[19])*(-217.89)+(r[20]-r[21])*(290.93)+      (r[22]-r[23])*(64.32)+(r[24]-r[25])*(56.08)+(r[26]-r[27])*(128.67))
   else:
      CalorDisociacion=0.0

   et1=5.0/2.0*K*T_bc+K*6325.0/(np.exp(6325.0/T_bc)-1.0)
   et2=3.0/2.0*K*T_bc
   et3=3.0/2.0*K*T_bc
   et4=5.0/2.0*K*T_bc+K*2273.0/(np.exp(2273.0/T_bc)-1.0)
   et5=5.0/2.0*K*T_bc+K*5370.0/(np.exp(5370.0/T_bc)-1.0)
   et6=6.0/2.0*K*T_bc+K*2295.0/(np.exp(2295.0/T_bc)-1.0)+K*5255.0/(np.exp(5255.0/T_bc)       -1.0)+K*5400.0/(np.exp(5400.0/T_bc)-1.0)
   # et7=3.0/2.0*K*T_bc
   # et8=5.0/2.0*K*T_bc+K*3445.0/(np.exp(3445.0/T_bc)-1.0)
   # et9=6.0/2.0*K*T_bc+K*4628.0/(np.exp(4628.0/T_bc)-1.0)+K*2152.0/(np.exp(2152.0/T_bc)       -1.0)+K*4746.0/(np.exp(4746.0/T_bc)-1.0)
   # et10=6.0/2.0*K*T_bc+K*4798.0/(np.exp(4798.0/T_bc)-1.0)+K*1366.0/(np.exp(1366.0/T_bc)        -1.0)+K*4951.0/(np.exp(4951.0/T_bc)-1.0)	  +K*4951.0/(np.exp(4951.0/T_bc)-1.0)+K*2339.0/(np.exp(2339.0/T_bc)-1.0)+          K*2339.0/(np.exp(2339.0/T_bc)-1.0)
   # et11=5.0/2.0*K*T_bc+K*4658.0/(np.exp(4658.0/T_bc)-1.0)+K*990.0/(np.exp(990.0/T_bc)-1.0)        +K*3253.0/(np.exp(3253.0/T_bc)-1.0)
   # et12=5.0/2.0*K*T_bc+K*2710.0/(np.exp(2710.0/T_bc)-1.0)
   # et13=6.0/2.0*K*T_bc+K*1878.0/(np.exp(1878.0/T_bc)-1.0)+K*1064.0/(np.exp(1064.0/T_bc)        -1.0)+K*2272.0/(np.exp(2272.0/T_bc)-1.0)
   # et14=5.0/2.0*K*T_bc+K*3098.0/(np.exp(3098.0/T_bc)-1.0)+K*822.0/(np.exp(822.0/T_bc)-1.0)        +K*1819.0/(np.exp(1819.0/T_bc)-1.0)
   # et15=5.0/2.0*K*T_bc+K*3895.0/(np.exp(3895.0/T_bc)-1.0)+K*2158.0/(np.exp(2158.0/T_bc)        -1.0)+K*2250.0/(np.exp(2250.0/T_bc)-1.0)
   # et16=5.0/2.0*K*T_bc+K*3350.0/(np.exp(3350.0/T_bc)-1.0)
   # et17=3.0/2.0*K*T_bc
   et18=6.0/2.0*K*T_bc
   et19=6.0/2.0*K*T_bc

   et_array = np.array([et1, et2, et3, et4, et5, et6, et18, et19])
   CalorTermico = -np.dot(et_array, dndt)

   # print(CalorTermico/-0.744445604930101)

   #ANTES:
   # CalorTermico=-(et1*dydt[Nvar2+1]+et2*dydt[Nvar2+2]+et3*dydt[Nvar2+3]+	       	et4*dydt[Nvar2+4]+et5*dydt[Nvar2+5]+et6*dydt[Nvar2+6]+et7*dydt[Nvar2+7]+	       	et8*dydt[Nvar2+8]+et9*dydt[Nvar2+9]+et10*dydt[Nvar2+10]+et11*dydt[Nvar2+11]+	       	et12*dydt[Nvar2+12]+et13*dydt[Nvar2+13]+et14*dydt[Nvar2+14]+et15*dydt[Nvar2+15]	       	+et16*dydt[Nvar2+16]+et17*dydt[Nvar2+17]+et18*dydt[Nvar2+18]+et19*dydt[Nvar2+19])


   det1=5.0/2.0*K+K*pow(6325.0,2.0)/(np.exp(6325.0/T_bc)-1.0)/(np.exp(6325.0/T_bc)-1.0)/T_bc/T_bc
   det2=3.0/2.0*K
   det3=3.0/2.0*K
   det4=5.0/2.0*K+K*2273.0*2273.0/(np.exp(2273.0/T_bc)-1.0)/(np.exp(2273.0/T_bc)-1.0)/T_bc/T_bc
   det5=5.0/2.0*K+K*5370.0*5370.0/(np.exp(5370.0/T_bc)-1.0)/(np.exp(5370.0/T_bc)-1.0)/T_bc/T_bc
   det6=6.0/2.0*K+K*2295.0*2295.0/(np.exp(2295.0/T_bc)-1.0)/(np.exp(2295.0/T_bc)-1.0)/T_bc/T_bc        +K*5255.0*5255.0/(np.exp(5255.0/T_bc)-1.0)/(np.exp(5255.0/T_bc)-1.0)/T_bc/T_bc+        K*5400.0*5400.0/(np.exp(5400.0/T_bc)-1.0)/(np.exp(5400.0/T_bc)-1.0)/T_bc/T_bc
   # det7=3.0/2.0*K
   # det8=5.0/2.0*K+K*3445.0*3445.0/(np.exp(3445.0/T_bc)-1.0)/(np.exp(3445.0/T_bc)-1.0)/T_bc/T_bc
   # det9=6.0/2.0*K+K*4628.0*4628.0/(np.exp(4628.0/T_bc)-1.0)/(np.exp(4628.0/T_bc)-1.0)/T_bc/T_bc        +K*2152.0*2152.0/(np.exp(2152.0/T_bc)-1.0)/(np.exp(2152.0/T_bc)-1.0)/T_bc/T_bc+        K*4746.0*4746.0/(np.exp(4746.0/T_bc)-1.0)/(np.exp(4746.0/T_bc)-1.0)/T_bc/T_bc
   # det10=6.0/2.0*K+K*4798.0*4798.0/(np.exp(4798.0/T_bc)-1.0)/(np.exp(4798.0/T_bc)-1.0)/T_bc/T_bc         +K*1366.0*1366.0/(np.exp(1366.0/T_bc)-1.0)/(np.exp(1366.0/T_bc)-1.0)/T_bc/T_bc         +K*4951.0*4951.0/(np.exp(4951.0/T_bc)-1.0)/(np.exp(4951.0/T_bc)-1.0)/T_bc/T_bc         +K*4951.0*4951.0/(np.exp(4951.0/T_bc)-1.0)/(np.exp(4951.0/T_bc)-1.0)/T_bc/T_bc+         K*2339.0*2339.0/(np.exp(2339.0/T_bc)-1.0)/(np.exp(2339.0/T_bc)-1.0)/T_bc/T_bc+         K*2339.0*2339.0/(np.exp(2339.0/T_bc)-1.0)/(np.exp(2339.0/T_bc)-1.0)/T_bc/T_bc
   # det11=5.0/2.0*K+K*4658.0*4658.0/(np.exp(4658.0/T_bc)-1.0)/(np.exp(4658.0/T_bc)-1.0)/T_bc/T_bc         +K*990.0*990.0/(np.exp(990.0/T_bc)-1.0)/(np.exp(990.0/T_bc)-1.0)/T_bc/T_bc+         K*3253.0*3253.0/(np.exp(3253.0/T_bc)-1.0)/(np.exp(3253.0/T_bc)-1.0)/T_bc/T_bc
   # det12=5.0/2.0*K+K*2710.0*2710.0/(np.exp(2710.0/T_bc)-1.0)/(np.exp(2710.0/T_bc)-1.0)/T_bc/T_bc
   # det13=6.0/2.0*K+K*1878.0*1878.0/(np.exp(1878.0/T_bc)-1.0)/(np.exp(1878.0/T_bc)-1.0)/T_bc/T_bc         +K*1064.0*1064.0/(np.exp(1064.0/T_bc)-1.0)/(np.exp(1064.0/T_bc)-1.0)/T_bc/T_bc+         K*2272.0*2272.0/(np.exp(2272.0/T_bc)-1.0)/(np.exp(2272.0/T_bc)-1.0)/T_bc/T_bc
   # det14=5.0/2.0*K+K*3098.0*3098.0/(np.exp(3098.0/T_bc)-1.0)/(np.exp(3098.0/T_bc)-1.0)/T_bc/T_bc+         K*822.0*822.0/(np.exp(822.0/T_bc)-1.0)/(np.exp(822.0/T_bc)-1.0)/T_bc/T_bc         +K*1819.0*1819.0/(np.exp(1819.0/T_bc)-1.0)/(np.exp(1819.0/T_bc)-1.0)/T_bc/T_bc
   # det15=5.0/2.0*K+K*3895.0*3895.0/(np.exp(3895.0/T_bc)-1.0)/(np.exp(3895.0/T_bc)-1.0)/T_bc/T_bc         +K*2158.0*2158.0/(np.exp(2158.0/T_bc)-1.0)/(np.exp(2158.0/T_bc)-1.0)/T_bc/T_bc         +K*2250.0*2250.0/(np.exp(2250.0/T_bc)-1.0)/(np.exp(2250.0/T_bc)-1.0)/T_bc/T_bc
   # det16=5.0/2.0*K+K*3350.0*3350.0/(np.exp(3350.0/T_bc)-1.0)/(np.exp(3350.0/T_bc)-1.0)/T_bc/T_bc
   # det17=3.0/2.0*K
   det18=6.0/2.0*K
   det19=6.0/2.0*K

   det_array = np.array([det1, det2, det3, det4, det5, det6, det18, det19])

   # if(t<=(duracion/2.0)):
   #    Pulso=Intensidad*t/(duracion/2.0)

   # else:
   #    if((t>=(duracion/2.0)) and (t<=duracion)):
   #       Pulso=(-2.0*Intensidad/duracion)*t+2.0*Intensidad
   #    else:
   #       Pulso=0.0
   Pulso = 0. #El valor de la variable Pulso no afecta en nada porque después se multiplica por cero

   # print(np.dot(det_array, n)/5.865651022892462e-13)

   # print((0.*Pulso  -  4.0*pi*R*R*v_R*pg   +   4.0*pi*R*R*mp*eh2o*Na/masa_H2O_kgmol   +    4.0*pi*R*R*qpp)/-11.277959686077713)
   # print(qpp, CalorDifusion, CalorDisociacion, CalorTermico, np.dot(det_array, n))
   dTdt=(0.*Pulso-4.0*pi*R*R*v_R*pg+4.0*pi*R*R*mp*eh2o*Na/masa_H2O_kgmol+4.0*pi*R*R*qpp         +CalorDifusion+CalorDisociacion+CalorTermico)/(np.dot(det_array, n))


   dT2dt=0.0

   dpgdt=Rg/(v-b)*dTdt+(-Rg*T_bc/pow(v-b,2.0)+2.0*a/pow(v,3.0))*dvdt+Rg*T_bc/pow(v-b,2.0)*dbdt-1.0/v/v*dadt

   dpg2dt=Rg/(v-b)*dT2dt


   drogdt=np.dot(masa_species_Kgmol, dndt)/Na/V-M/V/V*4.0*pi*R*R*v_R


   dydt[var_termo["R"]]=v_R 
   # dydt[4]=4.0*pi*R*R*(mp*Na/masa_H2O_kgmol); #Esto ya se hace en condensacion_y_evaporacion.ipynb

   dydt[var_termo["T_li"]]=0.0 #Acá está considerando que T_li no cambia en el tiempo

   #***************************************************************************
   #************************calculo de los coeficientes del Chebyshev**********
   #***************************************************************************

   # if((modelo == 3) || (modelo == 4)){

   #    Pp=dpgdt;

   #    f1=gama/(gama-1.0)*pg/T_bc/k;

   #    f2=2.0/R/R;

   #    f3=Pp/(3.0*(gama-1.0)*k*T_bc);

   #    f4=1.0/(T_bc*k*R*R);

   #    f5=gama*pg*v_R/(gama-1.0)/T_bc/k/R;

   #    f3=f3+f5;

   #    for(i=0;i<=NCh;i++)
   #       F[i]=0.0;

   #    #definicion del vector independiente

   #    j=0;
   #    while(j<=NCh){
   #       DPTCh=DerPCheb(colpoints[0],2*j);
   #       DSTCh=DerSCheb(colpoints[0],2*j);

   #       F[0]=F[0]+1.0/(f1*R*R)*y[6+j]*DSTCh;
   #       j=j+1;
   #    }

   #    cont=1;
   #    aux=4.0;
   #    sum=0.0;
   #    while(cont<=NCh){
   #       sum=sum+aux*y[6+cont];
   #       aux=pow(-1,cont)*(abs(aux)+(2*cont+1)*4.0);
   #       cont=cont+1;
   #    }

   #    F[0]=F[0]+Pp/f1+sum*f2/f1;

   #    for(i=1;i<NCh;i++){
   #       j=0;
   #       while(j<=NCh){
   #          DPTCh=DerPCheb(colpoints[i],2*j);
   #          DSTCh=DerSCheb(colpoints[i],2*j);

   #          F[i]=F[i]+1.0/(f1*R*R)*y[6+j]*DSTCh+f2/f1/colpoints[i]*y[6+j]*DPTCh+
   #          f3/f1*colpoints[i]*y[6+j]*DPTCh;

   #          j=j+1;

   #       }

   #       cont=0;
   #       sum=0.0;
   #       while(cont<=NCh){
   #          DPTCh=DerPCheb(colpoints[i],2*cont);
   #          sum=sum+y[6+cont]*DPTCh;
   #          cont=cont+1;
   #       }

   #       F[i]=F[i]-f4/f1*sum*sum+Pp/f1;


   #    }

   #    F[NCh]=0.0;

   #    for(i=0;i<=NCh;i++)
   #       x[i]=0.0;

   #    for(i=0;i<=NCh;i++){
   #       j=0;
   #       while(j<=NCh){
   #          x[i]=x[i]+Q[j][i]*F[j];
   #          j=j+1;
   #       }
   #    }

   #    for(i=0;i<=NCh;i++)
   #       dydt[6+i]=0.0;

   #    dydt[6+NCh]=x[NCh]/R[NCh][NCh];

   #    for(i=(NCh-1);i>=0;i--){
   #       sum=0;
   #       j=i+1;
   #       while(j<=(NCh)){
   #          sum=sum+R[i][j]*dydt[j+6];
   #          j=j+1;
   #       }
   #       dydt[i+6]=(x[i]-sum)/R[i][i];
   #    }
   # } #fin del if modelo igual al de Hao y Prosperetti


   #***************************************************************************
   #*******************los mpp*************************************************
   #***************************************************************************

   if(alfaM==0):
      mpp1=0.0
      mpp2=0.0
   else:
      pvap=pvap0
      dpvapdt=0.0

      #Asigno los valores de conductividad:
      k1 = np.zeros(n_species)
      k2 = np.zeros(n_species)
      Conductividad1(k1)
      Conductividad2(k2)

      dk1dt=  np.dot(k2, n)/ntot*dTdt-np.dot((k1+k2*T_bc),n)/ntot/ntot*dntdt+np.dot((k1 + k2*T_bc),dndt)/ntot

      dk2dt= np.dot(k2, n)/ntot*dT2dt

      dmdt=np.dot(masa_species_Kgmol,dndt)/ntot/Na
      dmdt=dmdt-m/ntot*dntdt

      dnp_dt=dntdt/V-4.0*3.14159*ntot*R*R*v_R/V/V

      dlandadt=4.0*3.14159*R*R*v_R/(np.sqrt(2.0)*sigmap*ntot)- V/(np.sqrt(2.0)*sigmap*ntot*ntot)*dntdt


      if(modelo == 1):
         dqdT_bi=k/n_Yasui/landa
         dq1dt=dk1dt*(T_bi-T_bc)/n_Yasui/landa-k/n_Yasui/landa*dTdt-k*(T_bi-T_bc)/n_Yasui/landa/landa*dlandadt
         dq2dt=dk2dt*(T_bi-T_bc)/n_Yasui/landa-k/n_Yasui/landa*dT2dt

      if(v_R == 0):
         dlth1dt=v_R/pi
         dlth2dt=0.0
      else:
         if((R/pi)<(np.sqrt(R*Xi/abs(v_R)))):
            dlth1dt=v_R/pi
            dlth2dt=0.0
         else:
            dlth1dt=0.5*np.sqrt(Xi/R/v_R)*v_R
            dlth2dt=-0.5*np.sqrt(R*Xi/v_R/v_R/v_R)

      if(modelo == 2):
         dqdT_bi=0.0
         dq1dt=dk1dt*(Tinf-T_bc)/lth-k/lth*dTdt-k/lth/lth*(Tinf-T_bc)*dlth1dt
         dq2dt=dk2dt*(Tinf-T_bc)/lth-k/lth*dT2dt-k/lth/lth*(Tinf-T_bc)*dlth2dt

      # if(modelo == 3){
      #    dqdT_bi=0.0;
      #    dq1dt=0.0;
      #    dq2dt=0.0;
      #    j=NCh;

      #    while(j>=0){
      #       DPTCh=DerPCheb(1,2*j);
      #       dq1dt=dq1dt-1.0/R/R*v_R*y[6+j]*DPTCh+1.0/R*dydt[6+j]*DPTCh;
      #       j=j-1;
      #    }

      # }

      # if(modelo == 4){

      #    dqdT_bi=0.0;
      #    dq1dt=0.0;
      #    dq2dt=0.0;
      #    j=NCh;

      #    while(j>=2){
      #       DPTCh=DerPCheb(1,2*j);
      #       dq1dt=dq1dt-1.0/R/R*v_R*y[6+j]*DPTCh+1.0/R*dydt[6+j]*DPTCh;
      #       j=j-1;
      #    }

      #    dq1dt=dq1dt+dk1dt*(Tinf-T_bc)/lth-k/lth*dTdt-k/lth/lth*(Tinf-T_bc)*dlth1dt;
      #    dq2dt=dq2dt+dk2dt*(Tinf-T_bc)/lth-k/lth*dT2dt-k/lth/lth*(Tinf-T_bc)*dlth2dt;

      # }


      dT_bi1dt=(dydt[var_termo["T_li"]]-(2.0-ap*alfae)/2.0/K/alfae*np.sqrt(pi/2.0/K)*(-1.0/np_/np_*dnp_dt*np.sqrt(m/T_bi)*qpp+  1.0/2.0/np_*np.sqrt(1.0/m/T_bi)*dmdt*qpp+1.0/np_*np.sqrt(m/T_bi)*dq1dt))/    (1.0+(2.0-ap*alfae)/2.0/K/alfae*np.sqrt(pi/2.0/K)*   (-1.0/2.0/np_*np.sqrt(m/T_bi/T_bi/T_bi)*qpp+1.0/np_*np.sqrt(m/T_bi)*dqdT_bi))

      dT_bi2dt=-(2.0-ap*alfae)/2.0/K/alfae*np.sqrt(pi/2.0/K)/np_*np.sqrt(m/T_bi)*dq2dt/   (1.0+(2.0-ap*alfae)/2.0/K/alfae*np.sqrt(pi/2.0/K)*  (-1.0/2.0/np_*np.sqrt(m/T_bi/T_bi/T_bi)*qpp+1.0/np_*np.sqrt(m/T_bi)*dqdT_bi))

      dpv1dt=(1.0-n[5]/ntot)*pg/ntot*dntdt+n[5]/ntot*dpgdt

      dpv2dt=n[5]/ntot*dpg2dt


      if(rapidez ==1):
         mpp1=alfaM/np.sqrt(2.0*pi*Rv)*(dpvapdt/np.sqrt(T_li)-0.5*pvap/pow(T_li,1.5)*dydt[var_termo["T_li"]]-      1.0/np.sqrt(T_bi)*dpv1dt+0.5*pv/pow(T_bi,1.5)*dT_bi1dt)

         mpp2=alfaM/np.sqrt(2.0*pi*Rv)*(-1.0/np.sqrt(T_bi)*dpv2dt+0.5*pv/pow(T_bi,1.5)*dT_bi2dt)

      else:
         Omega=mp/pv*np.sqrt(Rv*T_bc/2.0)

         Gama=np.exp(-Omega*Omega)-Omega*np.sqrt(3.14159)*(1.0-erf(Omega))

         mpp1=alfaM/np.sqrt(2.0*pi*Rv)*(dpvapdt/np.sqrt(T_li)-0.5*pvap/pow(T_li,1.5)*dydt[var_termo["T_li"]]-      mp/pv*(1.0-erf(Omega))*dpv1dt*np.sqrt(pi*Rv*T_bc/2.0/T_bi)+      0.5*mp*(1.0-erf(Omega))*np.sqrt(pi*Rv/2.0/T_bi/T_bc)*dTdt-      Gama/np.sqrt(T_bi)*dpv1dt+0.5*Gama*pv/pow(T_bi,1.5)*dT_bi1dt)/      (1.0-alfaM/2.0*(1.0-erf(Omega))*np.sqrt(T_bc/T_bi))

         mpp2=alfaM/np.sqrt(2.0*pi*Rv)*(-mp/pv*(1.0-erf(Omega))*dpv2dt*np.sqrt(pi*Rv*T_bc/      2.0/T_bi)+0.5*mp*(1.0-erf(Omega))*np.sqrt(pi*Rv/2.0/T_bi/T_bc)*dT2dt-      Gama/np.sqrt(T_bi)*dpv2dt+0.5*Gama*pv/pow(T_bi,1.5)*dT_bi2dt)/      (1.0-alfaM/2.0*(1.0-erf(Omega))*np.sqrt(T_bc/T_bi))


   #***************************************************************************
   #***************************************************************************
   #***************************************************************************

   dpB1dt=dpgdt+2.0*sigma*v_R/R/R+4.0*mu*v_R/R/R*(v_R-mp/rol)+      4.0*mu/R/rol*mpp1-2.0*mp*mpp1*(1.0/rol-1.0/rog)-mp*mp/rog/rog*drogdt

   dpB2dt=dpg2dt-4.0*mu/R+4.0*mu/R/rol*mpp2-2.0*mp*mpp2*(1.0/rol-1.0/rog)

   ps = 0
   # ps=Amp*sin(w*(t+R/cl)-fi); #presion acustica

   # dydt[4]=dpgdt #NO SÉ POR QUÉ ESTÁ ESTO ACÁ. LO SACO POR LAS DUDAS

   dydt[var_termo["v_R"]]=(1.0/rol*(1.0+v_R/cl)*(pB-ps-Pinf)+mp/rol*(v_R+mp/2.0/rol+mp*v_R/2.0/cl/rol)-         3.0/2.0*v_R*v_R*(1.0-1.0/3.0*v_R/cl+2.0*mp/3.0/cl/rol)+R/cl/rol*dpB1dt+         mpp1*R/rol*(1.0-v_R/cl+mp/cl/rol))/(R*(1.0-v_R/cl+mp/cl/rol)-         mpp2*R/rol*(1.0-v_R/cl+mp/cl/rol)-R/cl/rol*dpB2dt);

   dydt[var_termo["T_bc"]]=dTdt+dT2dt*dydt[var_termo["v_R"]]

   return

4